In [1]:
import pandas as pd
import numpy as np
from ambry import library
l = library()

p = l.get('ffiec.gov-hmda-2010e-lar-2010-ca').partition
str(p.identity.fqname)


'ffiec.gov-hmda-2010e-lar-2010-ca-0.0.7~p02X00X007'

In [2]:
ts = l.get('ffiec.gov-hmda-2010e-ts').partition.pandas

In [3]:
def get_ts(year):
    ts_name = ts[ts.activity_year == year][['respondent_id','respondent_name']].copy()
    ts_name.respondent_id = ts_name.respondent_id.astype(str)
    ts_name = ts_name.drop_duplicates(subset=['respondent_name']).set_index(['respondent_id'])
    return ts_name

def get_lar(year):
    # 'ffiec.gov-hmda-2010e-lar-2012-ca'
    lar =  (l.get('ffiec.gov-hmda-2010e-lar-{}-ca'.format(year)).partition
            .select("SELECT * FROM lar WHERE county_code = 73")
            .pandas )
    
    lar['tract'] = lar.census_tract_number # Save the trouble of changing column names later. 
    return lar

In [4]:
def generate_share_frame(df, ts_name, group):
    dfx = pd.DataFrame()
    dfx['count'] = df.groupby('respondent_id').count()['id']
    dfx['count_pct'] = np.round(dfx['count'] / dfx['count'].sum() * 100.0, 2)

    dfx['amount'] = df.groupby('respondent_id').sum()['loan_amount']
    dfx['amount_pct'] = np.round(dfx['amount'] / dfx['amount'].sum() * 100.0,2)

    dfx['amount_avg'] = np.round(df.groupby('respondent_id').mean()['loan_amount'],0)

    dfx['amount_median'] = np.round(df.groupby('respondent_id').median()['loan_amount'],0)
    
    dfx = (dfx.merge(ts_name,left_index = True, right_index = True)
               .set_index(['respondent_name'] )
               .sort('count_pct', ascending = False))
    
    # HACK. There is certainly a better way to set the first-level of a new multi index on columns,
    # but I'm tired of reading docs. 
    x = dfx.T
    x['group'] = group
    dfx =  x.set_index('group',append=True).reorder_levels([1,0]).T

    return dfx
    

In [5]:
def build_market_share_report(lar, year):
    ts_name = get_ts(year)
    all_share = generate_share_frame(lar,ts_name,'all')
    no_refi_share = generate_share_frame(lar[lar.loan_purpose != 2], ts_name, 'Excl. Home Imp.')
    combined = all_share.join(no_refi_share)
    return combined

In [7]:
import os
for year in (2010, 2011, 2012, 2013):
    lar = get_lar(year)
    df = build_market_share_report(lar, year)
    print year, len(df)
    
    path = 'market-share/{}/county/sandiego-county.csv'.format(year)
    
    d = os.path.dirname(path)
    if not os.path.exists(d):
        os.makedirs(d)
    
    df.to_csv(path)

2010 673
2011 661
2012 722
2013 722


In [8]:
places = pd.read_csv('../data/ca_tract_to_place.csv', delimiter='\t')
places = places[places.county == '06073']


In [93]:
for year in (2010, 2011, 2012, 2013):
    lar = get_lar(year)
    
    for placefp in list(places.placefp.unique()):
        
        larp = lar.merge(places, on='tract').copy()
        larp_sub = larp[larp.placefp == placefp]
        report = build_market_share_report(larp_sub, year)
        
        raw_place_name = larp_sub.placenm.unique()[0]
        place_name = raw_place_name.replace(' ','_').replace(',','').lower()
        
        place_type = 'other'
        
        if 'cdp' in place_name:
            place_type = 'cdp'
        elif 'city' in place_name:
            place_type = 'city'
        
        path = "market-share/{}/{}/{}.csv".format(year,place_type,place_name)
        report.columns.names = ['{}, {}'.format(raw_place_name, year), None]
        print path
        
        d = os.path.dirname(path)
        if not os.path.exists(d):
            os.makedirs(d)
        
        report.columns.set_levels([u'All Loans', u'Excluding Home Imp.'], level = 0, inplace = True)
        report.columns.set_levels([u'Total Amount', u'Average Loan Size', u'Median Loan Size', u'Market Share % ($)',
                           u'Count', u'Market Share % (#)'], level = 1, inplace = True)
        
        # Set a row number
        report['#'] = 0
        report['#'] = range(1,len(report['#'])+1)
        
        report_sum = report.drop('#', axis=1,level=0).sum()
        
        report = report.set_index('#', append=True).reorder_levels([1,0])[:30]
        
        report = pd.concat([report, pd.DataFrame( {(31,'Total'):report_sum }).T])
        
        report.to_csv(path)

        


market-share/2010/city/san_diego_city_ca.csv
market-share/2010/city/lemon_grove_city_ca.csv
market-share/2010/cdp/la_presa_cdp_ca.csv
market-share/2010/other/_.csv
market-share/2010/cdp/bonita_cdp_ca.csv
market-share/2010/city/national_city_city_ca.csv
market-share/2010/city/chula_vista_city_ca.csv
market-share/2010/city/del_mar_city_ca.csv
market-share/2010/city/imperial_beach_city_ca.csv
market-share/2010/city/coronado_city_ca.csv
market-share/2010/cdp/casa_de_oro-mount_helix_cdp_ca.csv
market-share/2010/cdp/spring_valley_cdp_ca.csv
market-share/2010/cdp/rancho_san_diego_cdp_ca.csv
market-share/2010/city/la_mesa_city_ca.csv
market-share/2010/city/el_cajon_city_ca.csv
market-share/2010/cdp/crest_cdp_ca.csv
market-share/2010/cdp/harbison_canyon_cdp_ca.csv
market-share/2010/cdp/granite_hills_cdp_ca.csv
market-share/2010/cdp/bostonia_cdp_ca.csv
market-share/2010/city/santee_city_ca.csv
market-share/2010/cdp/winter_gardens_cdp_ca.csv
market-share/2010/cdp/lakeside_cdp_ca.csv
market-share/